# Examples

Below you can find some examples of how to use the 🤖 `Megabots` library.


### Creating a bot

The `bot` object is the main object of the library. It is used to create a bot and to interact with it.

The `index` argument specifies the index to use for the bot. It can either be a saved index file (e.g., `index.pkl`) or a directory of documents (e.g., `./index`). In the case of the directory the index will be automatically created. If no index is specified the `bot` will look for `index.pkl` or `./index`.

In [1]:
from megabots import bot

qnabot = bot("qna-over-docs", index="./index.pkl")

qnabot.ask("what was the first roster of the avengers?")


/Users/momegas/Desktop/qnabot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...


'The first roster of the Avengers included Iron Man, Thor, Hulk, Ant-Man, and the Wasp.'

### Changing the bot's prompt

You can change the bot's promnpt to customize it to your needs. In the `qna-over-docs` type of bot you will need to pass 2 variables for the `context` (knwoledge searched from the index) and the `question` (the human question).


In [2]:
from megabots import bot

prompt = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer in the style of Tony Stark.

{context}

Question: {question}
Helpful humorous answer:"""

qnabot = bot("qna-over-docs", index="./index.pkl", prompt=prompt)

qnabot.ask("what was the first roster of the avengers?")


Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...


'The first roster of the Avengers included Iron Man, Thor, Hulk, Ant-Man, and the Wasp.'

### Using Megabots with Milvus

Megabots `bot` can also use Milvus as a backend for its search engine. You can find an example of how to do it below.

In order to run Milvus you need to follow [this guide](https://milvus.io/docs/example_code.md) to download a docker compose file and run it.
The command is:

```bash
wget https://raw.githubusercontent.com/milvus-io/pymilvus/v2.2.7/examples/hello_milvus.py
```

You can then [install Attu](https://milvus.io/docs/attu_install-docker.md) as a management tool for Milvus


In [3]:
from megabots import bot

# Attach a vectorstore by passing the name of the database. Default port for milvus is 19530 and default host is localhost
# Point it to your files directory so that it can index the files and add them to the vectorstore
bot = bot("qna-over-docs", index="./examples/files/", vectorstore="milvus")

bot.ask("what was the first roster of the avengers?")


Using model: gpt-3.5-turbo


'The first roster of the Avengers included Iron Man, Thor, Hulk, Ant-Man, and the Wasp.'

Or use the `vectorstore` factory function for more customisation


In [4]:
from megabots import bot, vectorstore

milvus = vectorstore("milvus", host="localhost", port=19530)

bot = bot("qna-over-docs", index="./examples/files/", vectorstore=milvus)


Using model: gpt-3.5-turbo


### Working with memory

You can easily add memory to your `bot` using the `memory` parameter. It accepts a string with the type of the memory to be used. This defaults to some sane dafaults.
Should you need more configuration, you can use the `memory` function and pass the type of memory and the configuration you need.


In [5]:
from megabots import bot

qnabot = bot("qna-over-docs", index="./index.pkl", memory="conversation-buffer")

print(qnabot.ask("who is iron man?"))
print(qnabot.ask("was he in the first roster?"))


Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...
Iron Man is a superhero character who is a member of the Avengers. He is known for his high-tech suit of armor and his alter ego, Tony Stark.
Yes, Iron Man was part of the original Avengers lineup.


Or using the `memory`factory function

In [6]:
from megabots import bot, memory

qnabot = bot(
    "qna-over-docs",
    index="./index.pkl",
    memory=memory("conversation-buffer-window", k=5),
)

print(qnabot.ask("who is iron man?"))
print(qnabot.ask("was he in the first roster?"))


Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...
Iron Man is a superhero character who is a member of the Avengers. He is known for his high-tech suit of armor and his alter ego, Tony Stark.
Yes, Iron Man was part of the original Avengers lineup.


NOTE: For the `qna-over-docs` bot, when using memory and passing your custom prompt, it is important to remember to pass one more variable to your custom prompt to facilitate for chat history. The variable name is `history`.


In [7]:
from megabots import bot

prompt = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

{history}
Human: {question}
AI:"""

qnabot = bot(
    "qna-over-docs",
    prompt=prompt,
    index="./index.pkl",
    memory="conversation-buffer",
)

print(qnabot.ask("who is iron man?"))
print(qnabot.ask("was he in the first roster?"))

Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...
Iron Man is a superhero character who is a member of the Avengers. He is a wealthy businessman named Tony Stark who uses his advanced technology to create a suit of armor that gives him superhuman abilities.
Yes, Iron Man was part of the original Avengers lineup.
